# 1유형

# 2유형

# 3유형

### 1) 정규성 검정

In [9]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/height1.csv')

# 정규성 검정
gA = df
from scipy.stats import shapiro, kstest, normaltest, anderson
print(shapiro(gA))
print(kstest(gA, "norm"))
print(normaltest(gA))

ShapiroResult(statistic=0.9872668981552124, pvalue=0.4558176100254059)
KstestResult(statistic=1.0, pvalue=0.0)
NormaltestResult(statistic=array([2.10749347]), pvalue=array([0.34862908]))


### 2) 등분산 검정

In [22]:
import pandas as pd 
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/scipy2.csv')
# df.info() # class, score
# print(df['class'].value_counts())

# 그룹나누기
gA = df.loc[df['class']=='A', 'score']
gB = df.loc[df['class']=='B', 'score']
# print(gA.shape, gB.shape)

# 등분산 검정
from scipy.stats import bartlett, levene
print(bartlett(gA, gB))
print(levene(gA, gB, center='median'))

BartlettResult(statistic=0.26035880448930865, pvalue=0.609873758447687)
LeveneResult(statistic=0.3145466542912649, pvalue=0.5751662820554713)


### 3) 모수 검정

In [4]:
# 단일표본 t검정
# 그리고 평균키는 165라 판단할수 있는지 귀무가설과 대립가설을 설정한 후 유의수준 5%로 검정하라.
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/height1.csv')

# 그룹 나누기
gA = df
n = 165

# 검정
from scipy.stats import ttest_1samp
s, p = ttest_1samp(gA, n, alternative='two-sided')
print(s, p)

[3.2017885] [0.00183672]


In [32]:
#  독립표본 t검정
# 두개 학급의 시험성적에 대한 데이터이다. 두 학습의 시험 평균(비모수검정의 경우 중위값)은 동일하다 말할 수 있는지 확인 하라
import pandas as pd 
df1 = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/ind1.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/ind2.csv')

# 그룹 나누기
gA = df1
gB = df2

from scipy.stats import ttest_ind
print(ttest_ind(gA, gB, alternative='two-sided', equal_var=True))

Ttest_indResult(statistic=array([2.76719074]), pvalue=array([0.00619015]))


In [26]:
# 대응표본 t검정 
# 특정 질병 집단의 투약 전후의 혈류량 변화를 나타낸 데이터이다. 투약 전후의 변화가 있는지 검정하라
import pandas as pd 
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/rel2.csv')

# 그룹 나누기 
gA = df['after']
gB = df['before']

# 검정
from scipy.stats import ttest_rel
print(ttest_rel(gA, gB, alternative='less'))

Ttest_relResult(statistic=2.5535473487670672, pvalue=0.9940366276377267)


In [ ]:
# one way ANOVA


# 그룹나누기
gA = 
gB = 
gC = 

# 검정
from scipy.stats import f_oneway
print(f_oneway(gA, gB, gC)) # 반환값 F-statistics, P-value

### 4) 비모수 검정 - 범주형

In [48]:
# 카이제곱 적합성 검정


# 그룹나누기
obs = 
exp = 

# 검정
from scipy.stats import chisquare
print(chisquare(exp, obs))

In [36]:
# 카이제곱 동질성 검정


# 그룹나누기
crstab = pd.crosstab(index= , columns=)

# 검정
from scipy.stats import chi2_contingency
print(chi2_contingency(crstab))

In [37]:
# 카이제곱 독립성 검정


# 그룹나누기
crstab = pd.crosstab(index= , columns=)

# 검정
from scipy.stats import chi2_contingency
print(chi2_contingency(crstab))

In [51]:
# 피셔의 정확성 검정

# 그룹나누기
# metrics = 

# 검정
from scipy.stats import fisher_exact
print(fisher_exact(metrics)) # 반환값 odd, p-value

['ConstantInputWarning', 'DegenerateDataWarning', 'FitError', 'NearConstantInputWarning', 'NumericalInverseHermite', 'alexandergovern', 'alpha', 'anderson', 'anderson_ksamp', 'anglit', 'ansari', 'arcsine', 'argus', 'barnard_exact', 'bartlett', 'bayes_mvs', 'bernoulli', 'beta', 'betabinom', 'betaprime', 'biasedurn', 'binned_statistic', 'binned_statistic_2d', 'binned_statistic_dd', 'binom', 'binom_test', 'binomtest', 'boltzmann', 'bootstrap', 'boschloo_exact', 'boxcox', 'boxcox_llf', 'boxcox_normmax', 'boxcox_normplot', 'bradford', 'brunnermunzel', 'burr', 'burr12', 'cauchy', 'chi', 'chi2', 'chi2_contingency', 'chisquare', 'circmean', 'circstd', 'circvar', 'combine_pvalues', 'contingency', 'cosine', 'cramervonmises', 'cramervonmises_2samp', 'crystalball', 'cumfreq', 'describe', 'dgamma', 'differential_entropy', 'dirichlet', 'distributions', 'dlaplace', 'dweibull', 'energy_distance', 'entropy', 'epps_singleton_2samp', 'erlang', 'expon', 'exponnorm', 'exponpow', 'exponweib', 'f', 'f_oneway

### 5) 비모수 검정 - 순서형

In [38]:
# 단일표본 검정


# 그룹나누기
gA = 
n

# 검정
from scipy.stats import wilcoxon
print(wilcoxon(gA-n))

In [41]:
# 쌍체 3표본 검정


# 그룹나누기
gA = 
gB = 

# 검정
from scipy.stats import wilcoxon
print(wilcoxon(gA-gB))

In [40]:
# 독립 2표본 검정

# 그룹나누기
gA = 
gB =

# 검정1
from scipy.stats import ransksums
print(ransksums(gA-gB))

# 검정2
from scipy.stats import mannwhitneyu
print(mannwhitneyu(gA, gB))

In [42]:
# kruskal 검정검정


# 그룹나누기
gA = 
gB = 
gC = 

# 검정
from scipy.stats import kruskal
print(kruskal(gA, gB, gC))  # 반환값 : H-statistics, P-value